# Müzik Türü Sınıflandırma Projesi

Bu notebook, FMA (Free Music Archive) veri setini kullanarak müzik türü sınıflandırma modeli geliştirmek için veri hazırlama ve dengeleme işlemlerini içermektedir.

## Gerekli Kütüphanelerin İçe Aktarılması
Aşağıdaki hücrede, projede kullanılacak temel Python kütüphaneleri import edilmektedir:

# Music Genre Classification with LSTM and BorderlineSMOTE

This notebook demonstrates a robust workflow for music genre classification using deep learning (LSTM) and advanced data balancing (BorderlineSMOTE). Key steps:
- Data loading and exploration
- Feature selection and scaling
- Class balancing with BorderlineSMOTE
- LSTM model training and evaluation
- Experiment logging, explainability, and creative enhancements

---

*All code and explanations are provided in both Turkish and English for clarity and accessibility.*


In [ ]:
# === CONFIGURATION ===
# All key parameters and paths in one place for reproducibility and easy tuning
DATA_DIR = 'fma_metadata'
TRACKS_PATH = f'{DATA_DIR}/tracks.csv'
FEATURES_PATH = f'{DATA_DIR}/features.csv'

# Feature Selection
K_BEST = 225  # Number of features to select

# Data Balancing
MIN_SAMPLES_THRESHOLD = 20  # For BorderlineSMOTE (used in RandomOverSampler step)

# LSTM Model & Training
SEQUENCE_LENGTH = 10  # Sequence length for LSTM
HIDDEN_SIZE = 64 # Hidden size of LSTM layers
NUM_LAYERS = 2     # Number of LSTM layers
DROPOUT_PROB = 0.3 # Dropout probability in LSTM
BIDIRECTIONAL = True # Whether to use a bidirectional LSTM
LEARNING_RATE = 0.001
EPOCHS = 100 # Max epochs (EarlyStopping will be used)
BATCH_SIZE = 512
PATIENCE_EARLY_STOPPING = 5 # Patience for early stopping

# General
RANDOM_STATE = 42
MODEL_SAVE_PATH = 'best_optimized_lstm.pth'
SCALER_SAVE_PATH = 'scaler.pkl'
ENCODER_SAVE_PATH = 'label_encoder.pkl'
LOG_FILE = 'experiment_log.json'

print('Configuration loaded.')
print(f"K_BEST: {K_BEST}, SEQUENCE_LENGTH: {SEQUENCE_LENGTH}, BATCH_SIZE: {BATCH_SIZE}")
print(f"LSTM: HIDDEN_SIZE={HIDDEN_SIZE}, NUM_LAYERS={NUM_LAYERS}, DROPOUT_PROB={DROPOUT_PROB}, BIDIRECTIONAL={BIDIRECTIONAL}")
print(f"TRAINING: LR={LEARNING_RATE}, EPOCHS={EPOCHS}, PATIENCE_EARLY_STOPPING={PATIENCE_EARLY_STOPPING}")


In [ ]:
# === ENVIRONMENT & PACKAGE VERSIONS ===
import sys
import platform
import sklearn
import torch
import imblearn
import pandas as pd
import numpy as np
import matplotlib
import seaborn

print('Python version:', sys.version)
print('Platform:', platform.platform())
print('scikit-learn:', sklearn.__version__)
print('PyTorch:', torch.__version__)
print('imbalanced-learn:', imblearn.__version__)
print('pandas:', pd.__version__)
print('numpy:', np.__version__)
print('matplotlib:', matplotlib.__version__)
print('seaborn:', seaborn.__version__)


In [ ]:
# === DEVICE SELECTION ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print('CUDA device name:', torch.cuda.get_device_name(0))


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE
from collections import Counter

%matplotlib inline
sns.set(style='whitegrid')

# === Helper Functions ===
# This section consolidates various utility functions used throughout the notebook
# for plotting, seeding, saving/loading artifacts, logging, and custom metrics.
# These functions depend on global variables defined in the Configuration cell (e.g., paths)
# and the Device Selection cell (e.g., `device`).

In [ ]:
# Helper function imports
import joblib
import json
from datetime import datetime
import random
from sklearn.metrics import f1_score, roc_auc_score # Added for advanced_metrics

# --- Plotting ---
def plot_class_distribution(y_numeric_labels, class_name_array, title):
    """Plots the distribution of classes.

    Args:
        y_numeric_labels: A pandas Series or numpy array of numeric class labels.
        class_name_array: An array or list of class names, where the index corresponds to the numeric label.
        title: The title for the plot.
    """
    names_map = {i: class_name_array[i] for i in range(len(class_name_array))}
    mapped_names_counts = pd.Series(y_numeric_labels).map(names_map).value_counts()

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=mapped_names_counts.index, y=mapped_names_counts.values, hue=mapped_names_counts.index, palette='viridis', legend=False)
    ax.set_title(title)
    ax.set_xlabel('Sınıf') # Class (Turkish)
    ax.set_ylabel('Sayı')  # Count (Turkish)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# --- PyTorch Model Training Utilities ---
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=PATIENCE_EARLY_STOPPING, verbose=False, delta=0, path=MODEL_SAVE_PATH):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model to {self.path} ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# --- Reproducibility ---
def set_seed(seed_value=RANDOM_STATE):
    """Sets the seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Global seed set to {seed_value}")

# --- Artifact Saving/Loading ---
def save_scaler_and_encoder(scaler_obj, encoder_obj, scaler_path_param=SCALER_SAVE_PATH, encoder_path_param=ENCODER_SAVE_PATH):
    """Saves scaler and label encoder objects."""
    joblib.dump(scaler_obj, scaler_path_param)
    joblib.dump(encoder_obj, encoder_path_param)
    print(f'Scaler saved to {scaler_path_param}, encoder saved to {encoder_path_param}')

def save_model(model_to_save, path_param=MODEL_SAVE_PATH):
    """Saves a PyTorch model state_dict."""
    torch.save(model_to_save.state_dict(), path_param)
    print(f"Model saved to {path_param}")

def load_model(model_instance, path_param=MODEL_SAVE_PATH):
    """Loads a PyTorch model state_dict into a model instance."""
    model_instance.load_state_dict(torch.load(path_param, map_location=device))
    model_instance.eval() # Set to evaluation mode
    print(f"Model loaded from {path_param} to {device}")
    return model_instance

# --- Experiment Logging ---
def log_experiment(params_dict, metrics_dict, filename_param=LOG_FILE):
    """Logs experiment parameters and metrics to a JSON file."""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "params": params_dict,
        "metrics": metrics_dict
    }
    try:
        with open(filename_param, "a") as f: # Append mode
            f.write(json.dumps(log_entry) + "\n")
        print(f"Experiment logged to {filename_param}")
    except Exception as e:
        print(f"Logging failed: {e}")

# --- Evaluation Metrics & Plotting ---
def plot_confusion_matrix_and_report(y_true_labels, y_pred_labels, class_names_list, title='Confusion Matrix'):
    """Plots confusion matrix and prints classification report."""
    cm = confusion_matrix(y_true_labels, y_pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names_list, yticklabels=class_names_list)
    plt.title(title)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()
    print('\nClassification Report:')
    print(classification_report(y_true_labels, y_pred_labels, labels=np.arange(len(class_names_list)), target_names=class_names_list, zero_division=0))

def advanced_metrics(y_true_labels, y_pred_labels, y_probabilities, num_classes_for_roc):
    """Calculates and prints weighted F1-score and ROC-AUC."""
    f1 = f1_score(y_true_labels, y_pred_labels, average='weighted', zero_division=0)
    print(f'Weighted F1-score: {f1:.4f}')
    try:
        if y_probabilities is not None and y_probabilities.ndim == 2 and y_probabilities.shape[0] == len(y_true_labels):
            unique_labels_in_true = np.unique(y_true_labels)
            if len(unique_labels_in_true) > 1:
                roc_auc = roc_auc_score(y_true_labels, y_probabilities, multi_class='ovr', average='weighted', labels=np.arange(num_classes_for_roc))
                print(f'ROC-AUC (OvR, Weighted): {roc_auc:.4f}')
            else:
                print('ROC-AUC not calculated: Only one class present in y_true.')
        else:
            print('ROC-AUC not calculated: y_probabilities are not in the correct format or not provided.')
    except ValueError as e:
        print(f'ROC-AUC calculation error: {e}')
    except Exception as e:
        print(f'An unexpected error occurred during ROC-AUC calculation: {e}')

print("Helper functions defined and ready.")


In [ ]:
# Set global random seed for reproducibility using value from config
set_seed(RANDOM_STATE)

## Veri Yükleme ve Ön İşleme

Bu bölümdeki fonksiyon:
- FMA metadata dosyalarını yükler
- Gerekli sütunları seçer
- Eksik verileri temizler
- Etiketleri kodlar
- Veriyi sayısal formata dönüştürür

# 3. VERİ SETİ

## 3.1. Tanım ve Temin

Bu çalışmada müzik türü sınıflandırma amacıyla **FMA (Free Music Archive) Small** veri seti kullanılmıştır. FMA, telif hakkı içermeyen binlerce müzik dosyasını barındıran bir açık veri kümesidir ve farklı boyutlarda (small, medium, large, full) sunulmaktadır. Bu projede kullanılan "FMA Small" versiyonu, toplam **8,000 adet ses parçası** içermekte ve her biri **30 saniyelik** örneklerden oluşmaktadır. Veri seti, M. Deffayet ve arkadaşları tarafından derlenen bir çalışmanın parçasıdır [8].

### Veri Seti Özellikleri:
- **Toplam Dosya Sayısı**: 8,000 adet müzik parçası
- **Dosya Süresi**: Her bir parça 30 saniye
- **Öznitelik Sayısı**: 518 önceden çıkarılmış öznitelik
- **Format**: CSV dosyaları (tracks.csv, features.csv)
- **Lisans**: Creative Commons lisansı ile özgürce kullanılabilir

## 3.2. Öznitelik Yapısı

FMA Small veri seti içinde her bir müzik parçasına ait **önceden çıkarılmış 518 öznitelik (feature)** bulunmaktadır. Bu öznitelikler Python ortamında `features.csv` dosyası aracılığıyla sağlanmakta ve sesin zaman-frekans alanındaki temsilini içermektedir.

Öznitelikler, içeriklerine göre aşağıdaki ana gruplara ayrılmaktadır:

### 3.2.1. Chroma Öznitelikleri (`chroma_`)
- **Amaç**: Tonalite ve akor bilgilerini temsil eder
- **İçerik**: Müzikal armoniyi yansıtır
- **Kullanım**: Müzik türleri arasındaki tonal farklılıkları yakalamak için kritik

### 3.2.2. MFCC Öznitelikleri (`mfcc_`)
- **Tam Adı**: Mel-Frequency Cepstral Coefficients
- **Amaç**: Sesin mel frekans alanındaki cepstral bileşenleri
- **Özellik**: Ses tanıma ve sınıflandırmada yaygın olarak kullanılır
- **Avantaj**: İnsan kulağının frekans algısını taklit eder

### 3.2.3. Spektral Öznitelikler (`spectral_`)
- **İçerik**: Spektral yoğunluk, enerji ve frekans dağılımları
- **Bilgi**: Sesin frekans alanındaki karakteristik özellikleri
- **Kullanım**: Müzik türlerinin frekans imzalarını ayırt etmek için

### 3.2.4. Tonnetz Öznitelikleri (`tonnetz_`)
- **Amaç**: Tonal merkezler ve armonik ilişkiler
- **İçerik**: Müzikal yapı ve harmoni bilgileri
- **Özellik**: Soyut müzikal kavramları sayısal veriye dönüştürür

## 3.3. Veri Setinin Avantajları

Bu özniteliklerin genişliği sayesinde, model sadece sesin fiziksel frekans özelliklerini değil; aynı zamanda **müzikal yapı ve armoni** gibi soyut bilgileri de öğrenebilmektedir. Bu kapsamlı öznitelik seti, müzik türü sınıflandırma görevinde yüksek performans elde etmemizi sağlamaktadır.

### Teknik Avantajlar:
- **Çok boyutlu temsil**: 518 farklı öznitelik ile zengin veri temsili
- **Önceden işlenmiş**: Manuel öznitelik çıkarımı gerektirmez
- **Standartlaştırılmış**: Tüm örnekler aynı format ve sürede
- **Dengeli kategorizasyon**: Farklı müzik türlerini kapsayan geniş etiket seti

## 3.4. Veri Seti Keşfi ve İnceleme

Aşağıdaki kod blokları ile FMA veri setinin yapısını, öznitelik dağılımını ve sınıf bilgilerini praktik olarak inceleyelim:

In [ ]:
# FMA veri setinin dosya yapısını kontrol et
import os

print("=== FMA Veri Seti Dosya Yapısı ===")
print(f"Metadata klasörü mevcut: {os.path.exists('fma_metadata')}")
print(f"Audio klasörü mevcut: {os.path.exists('fma_small')}")

if os.path.exists('fma_metadata'):
    metadata_files = os.listdir('fma_metadata')
    print(f"\nMetadata dosyaları ({len(metadata_files)} adet):")
    for file in sorted(metadata_files):
        file_path = os.path.join('fma_metadata', file)
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path) / (1024*1024)  # MB cinsinden
            print(f"  - {file}: {file_size:.2f} MB")

if os.path.exists('fma_small'):
    # Ses dosyalarının sayısını hesapla
    audio_count = 0
    for root, dirs, files in os.walk('fma_small'):
        audio_count += len([f for f in files if f.endswith('.mp3')])
    print(f"\nToplam ses dosyası sayısı: {audio_count}")

In [ ]:
# Features.csv dosyasının yapısını incele
if os.path.exists('fma_metadata/features.csv'):
    print("=== Features.csv Dosya Yapısı ===")
    
    # İlk birkaç satırı oku (çok seviyeli başlık yapısı)
    features_sample = pd.read_csv('fma_metadata/features.csv', nrows=5)
    print(f"Features dosyası boyutu: {features_sample.shape}")
    print(f"Sütun sayısı: {len(features_sample.columns)}")
    
    # Çok seviyeli başlık yapısını doğru şekilde oku
    features_full = pd.read_csv('fma_metadata/features.csv', index_col=0, header=[0,1])
    
    print(f"\nToplam örneklem sayısı: {len(features_full)}")
    print(f"Toplam öznitelik sayısı: {len(features_full.columns)}")
    
    # Öznitelik gruplarını analiz et
    feature_groups = {}
    for col in features_full.columns:
        if len(col) >= 2:
            group = col[0]  # İlk seviye (ana grup)
            if group not in feature_groups:
                feature_groups[group] = 0
            feature_groups[group] += 1
    
    print("\n=== Öznitelik Grupları ===")
    for group, count in sorted(feature_groups.items()):
        print(f"{group}: {count} öznitelik")
        
    # İlk birkaç öznitelik örneği
    print("\n=== Örnek Öznitelikler (İlk 10) ===")
    for i, col in enumerate(features_full.columns[:10]):
        print(f"{i+1}. {col}")
else:
    print("Features.csv dosyası bulunamadı!")

In [ ]:
# Tracks.csv dosyasını ve tür bilgilerini incele
if os.path.exists('fma_metadata/tracks.csv'):
    print("=== Tracks.csv ve Tür Bilgileri ===")
    
    # Tracks dosyasını oku
    tracks = pd.read_csv('fma_metadata/tracks.csv', index_col=0, header=[0,1])
    print(f"Tracks dosyası boyutu: {tracks.shape}")
    
    # Tür bilgilerini analiz et
    if ('track', 'genre_top') in tracks.columns:
        genres = tracks[('track', 'genre_top')].dropna()
        print(f"\nTür bilgisi olan parça sayısı: {len(genres)}")
        
        # Tür dağılımı
        genre_counts = genres.value_counts()
        print(f"\nToplam farklı tür sayısı: {len(genre_counts)}")
        
        print("\n=== Tür Dağılımı (İlk 10) ===")
        for genre, count in genre_counts.head(10).items():
            percentage = (count / len(genres)) * 100
            print(f"{genre}: {count} parça ({percentage:.1f}%)")
            
        # En az ve en çok örnekli türler
        print(f"\nEn çok örnekli tür: {genre_counts.index[0]} ({genre_counts.iloc[0]} parça)")
        print(f"En az örnekli tür: {genre_counts.index[-1]} ({genre_counts.iloc[-1]} parça)")
        
    else:
        print("Tür bilgisi sütunu bulunamadı!")
else:
    print("Tracks.csv dosyası bulunamadı!")

In [ ]:
# Öznitelik gruplarının detaylı analizi
if os.path.exists('fma_metadata/features.csv'):
    print("=== Öznitelik Grupları Detaylı Analiz ===")
    
    features = pd.read_csv('fma_metadata/features.csv', index_col=0, header=[0,1])
    
    # Her grup için öznitelik sayısını ve örnek isimleri göster
    detailed_groups = {}
    for col in features.columns:
        if len(col) >= 2:
            group = col[0]
            subfeature = col[1] if len(col) > 1 else 'unknown'
            
            if group not in detailed_groups:
                detailed_groups[group] = []
            detailed_groups[group].append(subfeature)
    
    print("\n=== Her Gruptaki Öznitelik Türleri ===")
    for group, subfeatures in sorted(detailed_groups.items()):
        print(f"\n{group.upper()} Grubu ({len(subfeatures)} öznitelik):")
        
        # Benzersiz alt öznitelik türlerini göster
        unique_subfeatures = list(set(subfeatures))
        for subf in sorted(unique_subfeatures)[:5]:  # İlk 5 tanesi
            count = subfeatures.count(subf)
            print(f"  - {subf}: {count} adet")
        
        if len(unique_subfeatures) > 5:
            print(f"  ... ve {len(unique_subfeatures) - 5} tür daha")
    
    print("\n=== Veri Seti Özeti ===")
    print(f"✓ Toplam müzik parçası: {len(features):,}")
    print(f"✓ Toplam öznitelik sayısı: {len(features.columns):,}")
    print(f"✓ Farklı öznitelik grubu: {len(detailed_groups)}")
    print(f"✓ Veri boyutu: {features.memory_usage(deep=True).sum() / (1024**2):.1f} MB")
    
    # Eksik veri kontrolü
    missing_data = features.isnull().sum().sum()
    print(f"✓ Eksik veri: {missing_data:,} hücre ({(missing_data/(len(features)*len(features.columns)))*100:.2f}%)")

In [ ]:
# Öznitelik gruplarının görsel analizi
import matplotlib.pyplot as plt
import seaborn as sns

if os.path.exists('fma_metadata/features.csv') and os.path.exists('fma_metadata/tracks.csv'):
    
    # Features ve tracks verilerini yükle
    features = pd.read_csv('fma_metadata/features.csv', index_col=0, header=[0,1])
    tracks = pd.read_csv('fma_metadata/tracks.csv', index_col=0, header=[0,1])
    
    # Öznitelik gruplarının dağılımını görselleştir
    feature_groups = {}
    for col in features.columns:
        if len(col) >= 2:
            group = col[0]
            if group not in feature_groups:
                feature_groups[group] = 0
            feature_groups[group] += 1
    
    # Grafik 1: Öznitelik gruplarının dağılımı
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    groups = list(feature_groups.keys())
    counts = list(feature_groups.values())
    colors = plt.cm.Set3(np.linspace(0, 1, len(groups)))
    
    bars = plt.bar(groups, counts, color=colors)
    plt.title('Öznitelik Gruplarının Dağılımı', fontsize=14, fontweight='bold')
    plt.xlabel('Öznitelik Grubu')
    plt.ylabel('Öznitelik Sayısı')
    plt.xticks(rotation=45)
    
    # Bar üzerinde değerleri göster
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                str(count), ha='center', va='bottom', fontweight='bold')
    
    # Grafik 2: Tür dağılımı (eğer mevcut ise)
    plt.subplot(1, 2, 2)
    if ('track', 'genre_top') in tracks.columns:
        genres = tracks[('track', 'genre_top')].dropna()
        top_genres = genres.value_counts().head(8)
        
        colors_genre = plt.cm.Paired(np.linspace(0, 1, len(top_genres)))
        bars = plt.bar(range(len(top_genres)), top_genres.values, color=colors_genre)
        plt.title('En Yaygın 8 Müzik Türü', fontsize=14, fontweight='bold')
        plt.xlabel('Müzik Türü')
        plt.ylabel('Parça Sayısı')
        plt.xticks(range(len(top_genres)), top_genres.index, rotation=45, ha='right')
        
        # Bar üzerinde değerleri göster
        for bar, count in zip(bars, top_genres.values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
                    str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Veri seti görselleştirme tamamlandı!")
else:
    print("❌ Gerekli dosyalar bulunamadı - görselleştirme yapılamıyor.")

# 4. ÖZNİTELİK SEÇİMİ (FEATURE SELECTION)

## 4.1. Öznitelik Seçiminin Gerekliliği

Makine öğrenmesi projelerinde yüksek boyutlu veri setleriyle çalışmak, hem **hesaplama yükünü artırmakta** hem de modelin öğrenme sürecinde **aşırı uyum (overfitting)** riskini beraberinde getirmektedir. Bu nedenle, bu projede **öznitelik seçimi (feature selection)** işlemi uygulanarak sadece en bilgilendirici öznitelikler modele dahil edilmiştir.

### Yüksek Boyutluluk Problemleri:
- **Hesaplama Karmaşıklığı**: 518 öznitelik işlem süresini önemli ölçüde artırır
- **Aşırı Uyum Riski**: Gereksiz öznitelikler modelin genelleme kabiliyetini azaltır
- **Gürültü Etkisi**: İlgisiz öznitelikler sınıflandırma performansını olumsuz etkiler
- **Bellek Kullanımı**: Yüksek boyutlu veriler daha fazla bellek gerektirir

## 4.2. SelectKBest Yöntemi

FMA Small veri seti, her bir müzik parçası için toplam **518 öznitelik** içermektedir. Ancak bu özniteliklerin tamamının sınıflandırma açısından eşit öneme sahip olmadığı bilinmektedir. Bu nedenle, **SelectKBest yöntemi** kullanılarak en etkili **225 öznitelik** seçilmiştir.

### Yöntem Detayları:
- **Algoritma**: SelectKBest (Scikit-learn)
- **Skorlama Fonksiyonu**: f_classif (ANOVA F-test)
- **Seçilen Öznitelik Sayısı**: 225 (orijinal 518'den)
- **Boyut Azaltma Oranı**: %56.6 azaltma

### f_classif Skorlama Fonksiyonu:
Bu işlem sırasında **f_classif skorlama fonksiyonu** kullanılmış ve her özniteliğin sınıf ayrımına katkısı istatistiksel olarak ölçülmüştür. f_classif, her öznitelik için ANOVA F-test değeri hesaplayarak:
- Sınıflar arası varyansı sınıf içi varyansa oranlar
- Yüksek F-skoru = Daha iyi sınıf ayrım gücü
- Düşük p-değeri = İstatistiksel olarak anlamlı öznitelik

## 4.3. Beklenen Faydalar

Bu süreç sonucunda oluşturulan eğitim verisi:
- ✅ **Hesaplama açısından daha verimli** hale getirilmiş
- ✅ **Bilgi yoğunluğu artırılmış** bir yapı kazanmış
- ✅ **Aşırı uyum riskini azaltmış**
- ✅ **Model genelleme kabiliyetini artırmış**

## 4.4. Öznitelik Seçimi Uygulaması

Aşağıdaki kod blokları ile öznitelik seçimi işlemini gerçekleştirip, sonuçları analiz edelim:

In [ ]:
# Öznitelik seçimi için gerekli kütüphaneler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

# Verinin yüklenmesi ve ön işlenmesi
def load_and_preprocess_data():
    """
    FMA veri setini yükler ve öznitelik seçimi için hazırlar.
    Uses TRACKS_PATH and FEATURES_PATH from the global configuration.
    """
    print("=== Veri Yükleme ve Ön İşleme (Global Config Paths) ===")
    
    # Dosya varlığı kontrolü (using global config)
    if not os.path.exists(TRACKS_PATH) or not os.path.exists(FEATURES_PATH):
        raise FileNotFoundError(f"Gerekli veri dosyaları bulunamadı! Paths: {TRACKS_PATH}, {FEATURES_PATH}")
    
    # Tracks ve features dosyalarını yükle
    tracks = pd.read_csv(TRACKS_PATH, index_col=0, header=[0,1])
    features_df = pd.read_csv(FEATURES_PATH, index_col=0, header=[0,1]) # Renamed to avoid conflict
    
    # Statistics sütunlarını kaldır ve sayısal formata dönüştür
    # Ensure features_df is used here
    features_df = features_df.loc[:, features_df.columns.get_level_values(0) != 'statistics']
    features_df = features_df.astype(np.float32)
    
    # İndeksleri string formatına dönüştür
    features_df.index = features_df.index.astype(str)
    tracks.index = tracks.index.astype(str)
    
    # Tür bilgilerini al
    genre_series = tracks[('track', 'genre_top')].dropna()
    
    # Ortak indeksleri bul
    common_index = features_df.index.intersection(genre_series.index)
    
    # Veriyi filtrele
    X = features_df.loc[common_index]
    y_labels = genre_series.loc[common_index]
    
    # Eksik ve sonsuz değerleri temizle
    X = X.fillna(0).replace([np.inf, -np.inf], 0).astype(np.float32)
    
    # Etiketleri sayısallaştır
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)
    
    print(f"✅ Orijinal veri boyutu (X): {X.shape}")
    print(f"✅ Toplam sınıf sayısı: {len(label_encoder.classes_)}")
    print(f"✅ Sınıflar: {', '.join(label_encoder.classes_)}")
    
    return X, y, label_encoder, features_df.columns # Return original X, y, encoder, and original feature columns

# Veriyi yükle
X_original, y_original_labels, label_encoder_global, original_feature_columns = load_and_preprocess_data()

In [ ]:
# Öznitelik seçimi uygulama
print("=== Öznitelik Seçimi (SelectKBest) ===")

# SelectKBest ile en iyi K_BEST özniteliği seç (K_BEST from global config)
selector = SelectKBest(score_func=f_classif, k=K_BEST)

# Öznitelik seçimini uygula (using X_original and y_original_labels from fc93bb40)
X_selected = selector.fit_transform(X_original, y_original_labels) # Use y_original_labels (numeric) for fitting

# Seçilen özniteliklerin indekslerini al
selected_features_mask = selector.get_support()
# Use original_feature_columns from fc93bb40
selected_feature_names = original_feature_columns[selected_features_mask] 

# F-skorlarını al
f_scores = selector.scores_
selected_f_scores = f_scores[selected_features_mask]

print(f"\n📊 Öznitelik Seçimi Sonuçları:")
print(f"   • Orijinal öznitelik sayısı: {X_original.shape[1]}")
print(f"   • Seçilen öznitelik sayısı: {X_selected.shape[1]} (K_BEST={K_BEST})")
print(f"   • Boyut azaltma oranı: {((X_original.shape[1] - X_selected.shape[1]) / X_original.shape[1] * 100):.1f}%")
print(f"   • Veri boyutu değişimi: {X_original.shape} → {X_selected.shape}")

# En yüksek F-Skorlu öznitelikleri göster
print(f"\n🏆 En Yüksek F-Skorlu 10 Öznitelik:")
# Sort selected_f_scores and get top indices from that sorted list
sorted_indices_selected = np.argsort(selected_f_scores)[::-1] # Sort descending
top_indices_selected = sorted_indices_selected[:10]

for i, idx in enumerate(top_indices_selected):
    feature_name = selected_feature_names[idx]
    score = selected_f_scores[idx]
    print(f"   {i+1:2d}. {feature_name}: {score:.2f}")

# X_selected and selected_feature_names are now available for downstream use.

In [ ]:
# Seçilen özniteliklerin grup analizi
print("\n=== Seçilen Özniteliklerin Grup Analizi ===")

# Orijinal ve seçilen özniteliklerin grup dağılımını karşılaştır
original_groups = {}
selected_groups = {}

# Orijinal öznitelik gruplarını say (using original_feature_columns from fc93bb40)
for col in original_feature_columns:
    if len(col) >= 2:
        group = col[0]
        if group not in original_groups:
            original_groups[group] = 0
        original_groups[group] += 1

# Seçilen öznitelik gruplarını say (using selected_feature_names from 980a711c)
for col in selected_feature_names:
    if len(col) >= 2:
        group = col[0]
        if group not in selected_groups:
            selected_groups[group] = 0
        selected_groups[group] += 1

# Karşılaştırma tablosu oluştur
comparison_data = []
for group in original_groups.keys():
    original_count = original_groups[group]
    selected_count = selected_groups.get(group, 0)
    selection_rate = (selected_count / original_count) * 100 if original_count > 0 else 0
    
    comparison_data.append({
        'Öznitelik Grubu': group,
        'Orijinal Sayı': original_count,
        'Seçilen Sayı': selected_count,
        'Seçim Oranı (%)': f"{selection_rate:.1f}%"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n📋 Öznitelik Grupları Karşılaştırması:")
print(comparison_df.to_string(index=False))

# Hangi grupların daha çok seçildiğini analiz et
print("\n🎯 Grup Seçim Analizi:")
for _, row in comparison_df.iterrows():
    group = row['Öznitelik Grubu']
    rate = float(row['Seçim Oranı (%)'].replace('%', ''))
    if rate >= 50:
        print(f"   ✅ {group}: Yüksek seçim oranı ({rate:.1f}%) - Bu grup sınıflandırma için önemli")
    elif rate >= 30:
        print(f"   🔶 {group}: Orta seçim oranı ({rate:.1f}%) - Kısmen bilgilendirici")
    else:
        print(f"   ❌ {group}: Düşük seçim oranı ({rate:.1f}%) - Sınıflandırma için az önemli")

In [ ]:
# Öznitelik seçimi sonuçlarının görselleştirilmesi
print("\n=== Öznitelik Seçimi Görselleştirme ===")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Öznitelik Seçimi Analiz Sonuçları', fontsize=16, fontweight='bold')

# 1. F-skorlarının dağılımı
ax1 = axes[0, 0]
ax1.hist(f_scores, bins=50, alpha=0.7, color='lightblue', edgecolor='black')
# Use K_BEST from config for the threshold line
ax1.axvline(np.sort(f_scores)[-K_BEST], color='red', linestyle='--', linewidth=2, label=f'Seçim Eşiği (Top {K_BEST})')
ax1.set_title('F-Skorlarının Dağılımı')
ax1.set_xlabel('F-Skoru')
ax1.set_ylabel('Frekans')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Öznitelik gruplarının karşılaştırması
ax2 = axes[0, 1]
groups = list(original_groups.keys())
x_pos = np.arange(len(groups))
original_counts = [original_groups[g] for g in groups]
selected_counts = [selected_groups.get(g, 0) for g in groups]

width = 0.35
ax2.bar(x_pos - width/2, original_counts, width, label='Orijinal', alpha=0.7, color='lightcoral')
ax2.bar(x_pos + width/2, selected_counts, width, label='Seçilen', alpha=0.7, color='lightgreen')
ax2.set_title('Öznitelik Grupları: Orijinal vs Seçilen')
ax2.set_xlabel('Öznitelik Grubu')
ax2.set_ylabel('Öznitelik Sayısı')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(groups, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. En yüksek F-skorlu öznitelikler
ax3 = axes[1, 0]
top_n = 15
# Use selected_f_scores and selected_feature_names from cell 980a711c
sorted_indices_selected_viz = np.argsort(selected_f_scores)[-top_n:] # Get indices of top N from selected_f_scores
top_scores_viz = selected_f_scores[sorted_indices_selected_viz]
top_names_viz = [str(selected_feature_names[i]) for i in sorted_indices_selected_viz]

# Öznitelik isimlerini kısalt
top_names_short = [name.replace('(', '').replace(')', '').replace(',', '_')[:20] + '...' if len(str(name)) > 23 else str(name) for name in top_names_viz]

y_pos = np.arange(len(top_names_short))
colors = plt.cm.viridis(np.linspace(0, 1, len(top_scores_viz)))
ax3.barh(y_pos, top_scores_viz, color=colors)
ax3.set_title(f'En Yüksek F-Skorlu {top_n} Öznitelik')
ax3.set_xlabel('F-Skoru')
ax3.set_yticks(y_pos)
ax3.set_yticklabels(top_names_short, fontsize=8)
ax3.grid(True, alpha=0.3)

# 4. Boyut azaltma etkisi
ax4 = axes[1, 1]
labels = ['Orijinal Boyut', 'Seçilen Boyut']
sizes = [X_original.shape[1], X_selected.shape[1]]
colors = ['#ff9999', '#66b3ff']
explode = (0, 0.1)  # Seçilen kısmı vurgula

ax4.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax4.set_title('Boyut Azaltma Etkisi')

plt.tight_layout()
plt.show()

print("\n✅ Öznitelik seçimi görselleştirmesi tamamlandı!")

In [ ]:
# Öznitelik seçiminin etkilerinin detaylı analizi
print("\n=== Öznitelik Seçimi Etki Analizi ===")

# Bellek kullanımı karşılaştırması
original_memory = X_original.memory_usage(deep=True).sum() / (1024**2)  # MB
selected_memory = pd.DataFrame(X_selected).memory_usage(deep=True).sum() / (1024**2)  # MB
memory_reduction = ((original_memory - selected_memory) / original_memory) * 100

print(f"\n💾 Bellek Kullanımı Analizi:")
print(f"   • Orijinal veri bellek kullanımı: {original_memory:.1f} MB")
print(f"   • Seçilen veri bellek kullanımı: {selected_memory:.1f} MB")
print(f"   • Bellek tasarrufu: {memory_reduction:.1f}% ({original_memory-selected_memory:.1f} MB)")

# Hesaplama karmaşıklığı tahmini
original_complexity = X_original.shape[0] * X_original.shape[1]
selected_complexity = X_selected.shape[0] * X_selected.shape[1]
complexity_reduction = ((original_complexity - selected_complexity) / original_complexity) * 100

print(f"\n⚡ Hesaplama Karmaşıklığı:")
print(f"   • Orijinal işlem sayısı: {original_complexity:,}")
print(f"   • Seçilen işlem sayısı: {selected_complexity:,}")
print(f"   • Hesaplama azalması: {complexity_reduction:.1f}%")

# En bilgilendirici öznitelik gruplarını belirle
print(f"\n🎯 En Bilgilendirici Öznitelik Grupları:")
group_importance = {}
for i, col in enumerate(selected_feature_names):
    if len(col) >= 2:
        group = col[0]
        score = selected_f_scores[i]
        if group not in group_importance:
            group_importance[group] = []
        group_importance[group].append(score)

# Her grup için ortalama F-skoru hesapla
group_avg_scores = {}
for group, scores in group_importance.items():
    group_avg_scores[group] = np.mean(scores)

# Grupları önem sırasına göre sırala
sorted_groups = sorted(group_avg_scores.items(), key=lambda x: x[1], reverse=True)

for i, (group, avg_score) in enumerate(sorted_groups):
    count = len(group_importance[group])
    selection_rate = (count / original_groups[group]) * 100
    print(f"   {i+1}. {group}: Ort. F-skoru={avg_score:.2f}, Seçilen={count}/{original_groups[group]} ({selection_rate:.1f}%)")

print(f"\n📈 Özet:")
print(f"   ✅ Veri boyutu {X_original.shape[1]} → {X_selected.shape[1]} özniteliğe düşürüldü")
print(f"   ✅ {memory_reduction:.1f}% bellek tasarrufu sağlandı")
print(f"   ✅ {complexity_reduction:.1f}% hesaplama karmaşıklığı azaltıldı")
print(f"   ✅ En bilgilendirici grup: {sorted_groups[0][0]} (Ort. F-skoru: {sorted_groups[0][1]:.2f})")
print(f"   ✅ Model aşırı uyum riski azaltıldı")
print(f"   ✅ Eğitim süresi optimize edildi")

## 4.5. Sonraki Adımlar

Öznitelik seçimi işlemi tamamlandıktan sonra, veri seti şu şekilde hazır hale gelmiştir:

### ✅ Tamamlanan İşlemler:
1. **Boyut Azaltma**: 518 → 225 öznitelik
2. **Etiket Kodlama**: Müzik türleri sayısallaştırıldı
3. **Veri Temizleme**: Eksik ve sonsuz değerler giderildi
4. **Optimizasyon**: Bellek ve hesaplama verimliliği artırıldı

### 🔄 Sıradaki İşlemler:
1. **Veri Dengeleme**: RandomOverSampler + BorderlineSMOTE
2. **Eğitim-Test Bölme**: Stratified split uygulaması
3. **Model Eğitimi**: LSTM ağ yapısı
4. **Performans Değerlendirme**: Accuracy, precision, recall metrikleri

Veri seti artık **sınıf dengesizliği problemi çözülmek üzere** bir sonraki aşamaya hazırdır.

## Başlangıç Veri Analizi

# Veriyi yükle ve önişle - This cell now relies on variables from cell fc93bb40
# X_original, y_original_labels, label_encoder_global, original_feature_columns are already loaded

print("Plotting initial class distribution using data from 'load_and_preprocess_data'...")
# y_original_labels is numeric here, label_encoder_global.classes_ provides the names
plot_class_distribution(y_original_labels, label_encoder_global.classes_, 'Başlangıç Sınıf Dağılımı (Filtrelenmemiş)')

In [ ]:
# Veriyi böl ve eğitim dağılımını göster
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_original_labels, test_size=0.2, stratify=y_original_labels, random_state=RANDOM_STATE
)

plot_class_distribution(y_train, label_encoder_global.classes_, 'Eğitim Seti Dağılımı')
print(f'Eğitim/test bölünmesi tamamlandı: X_train {X_train.shape}, X_test {X_test.shape}')

# Detaylı dağılımı yazdır
unique, counts = np.unique(y_train, return_counts=True)
print("\nEğitim Seti Dağılımı (ham sayılar):")
for i, (u, c) in enumerate(zip(unique, counts)):
    print(f"Sınıf {u} ({label_encoder_global.classes_[i]}): {c} örnek")

## Veri Dengeleme - Aşama 1

İlk aşamada, çok az örneğe sahip sınıflar için RandomOverSampler kullanılıyor. Bu aşama, BorderlineSMOTE için yeterli örnek sayısına ulaşmamızı sağlar.

In [ ]:
# Adım 1: En az temsil edilen sınıflar için RandomOverSampler
print('\nAdım 1: Aşırı az temsil edilen sınıflar için RandomOverSampler uygulanıyor...')

# Mevcut sınıf dağılımını kontrol et
unique_train, counts_train = np.unique(y_train, return_counts=True)
print("\nEğitim setindeki mevcut dağılım:")
for i, (u, c) in enumerate(zip(unique_train, counts_train)):
    class_name = label_encoder_global.classes_[u]
    print(f"Sınıf {u} ({class_name}): {c} örnek")

# MIN_SAMPLES_THRESHOLD'dan az örneğe sahip sınıfları belirle
classes_to_oversample = {}
for i, (u, c) in enumerate(zip(unique_train, counts_train)):
    if c < MIN_SAMPLES_THRESHOLD:
        classes_to_oversample[u] = MIN_SAMPLES_THRESHOLD
        class_name = label_encoder_global.classes_[u]
        print(f"Sınıf {u} ({class_name}) örneklem sayısı {c} → {MIN_SAMPLES_THRESHOLD} artırılacak")

# Eğer örneklem artırılacak sınıf varsa RandomOverSampler uygula
if classes_to_oversample:
    ros = RandomOverSampler(sampling_strategy=classes_to_oversample, random_state=RANDOM_STATE)
    X_partial, y_partial = ros.fit_resample(X_train, y_train)
    print(f"\nRandomOverSampler uygulandı: {len(classes_to_oversample)} sınıf için örneklem artırıldı")
else:
    X_partial, y_partial = X_train, y_train
    print(f"\nTüm sınıflar zaten {MIN_SAMPLES_THRESHOLD} eşiğinin üzerinde, RandomOverSampler uygulanmadı")

# Ara sonuçları göster
unique_partial, counts_partial = np.unique(y_partial, return_counts=True)
print("\nRandomOverSampler sonrası dağılım (ham sayılar):")
for i, (u, c) in enumerate(zip(unique_partial, counts_partial)):
    # Use label_encoder_global from cell 20 for class names
    print(f"Sınıf {u} ({label_encoder_global.classes_[i]}): {c} örnek")

# Use label_encoder_global from cell 20 for class names
plot_class_distribution(y_partial, label_encoder_global.classes_, 'RandomOverSampler Sonrası')

## Veri Dengeleme - Aşama 2

İkinci aşamada, daha sofistike bir yaklaşım olan BorderlineSMOTE kullanılarak kalan sınıflar dengeleniyor. Bu yöntem, sadece rastgele kopyalama yerine sentetik örnekler oluşturur.

Not: Bu aşama, veri setinin yapısına bağlı olarak başarısız olabilir. Bu durumda, ilk aşamadaki sonuçlar kullanılacaktır.

In [ ]:
# Adım 2: Kalan sınıflar için BorderlineSMOTE
print('\nAdım 2: Kalan sınıflar için BorderlineSMOTE uygulanıyor...')
borderline_smote = BorderlineSMOTE(random_state=RANDOM_STATE) # Use RANDOM_STATE from config

try:
    X_res, y_res = borderline_smote.fit_resample(X_partial, y_partial)
    print(f'Kombine örnekleme tamamlandı: X_res {X_res.shape}, y_res {y_res.shape}')
    
    # Son dağılımı yazdır ve göster
    unique_res, counts_res = np.unique(y_res, return_counts=True)
    print("\nSon Dağılım (ham sayılar):")
    for i, (u, c) in enumerate(zip(unique_res, counts_res)):
        # Use label_encoder_global from cell 20 for class names
        print(f"Sınıf {u} ({label_encoder_global.classes_[i]}): {c} örnek")
    
    # Use label_encoder_global from cell 20 for class names
    plot_class_distribution(y_res, label_encoder_global.classes_, 'Son Dengelenmiş Dağılım')

except Exception as e:
    print(f'BorderlineSMOTE örnekleme başarısız oldu: {e} - kısmi örneklenmiş veri kullanılıyor')
    X_res, y_res = X_partial, y_partial
    # Use label_encoder_global from cell 20 for class names
    plot_class_distribution(y_res, label_encoder_global.classes_, 'Kısmi Örnekleme (BorderlineSMOTE başarısız)')

print("\nİşlem hattı tamamlandı. Yeniden örneklenmiş eğitim verisi (X_res, y_res) ve test verisi (X_test, y_test) hazır.")

## PyTorch LSTM MODEL EĞİTİMİ

In [ ]:
print("\nPyTorch LSTM Model Eğitimi Başlıyor...")

# Veri yükleme, önişleme, bölme ve dengeleme adımlarının tamamlandığı varsayılır.
# Bu noktada aşağıdaki değişkenlerin mevcut olması beklenir:
# X_res, y_res (Dengelenmiş eğitim verisi)
# X_test, y_test (Test verisi)
# label_encoder_global (LabelEncoder nesnesi)
# X_val, y_val (Doğrulama verisi - bir sonraki hücrede oluşturulacak)

In [ ]:
# Dengelenmiş veri setinden doğrulama seti ayır
X_train_bal, X_val, y_train_bal, y_val = train_test_split(
    X_res, y_res, test_size=0.1, stratify=y_res, random_state=RANDOM_STATE # Use RANDOM_STATE from config
)

# Veri Ölçeklendirme (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_bal)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test) # Ensure X_test from the initial split is used here

print("Veri ölçeklendirme tamamlandı.")
print(f"Ölçeklenmiş eğitim verisi boyutu: {X_train_scaled.shape}")
print(f"Ölçeklenmiş doğrulama verisi boyutu: {X_val_scaled.shape}")
print(f"Ölçeklenmiş test verisi boyutu: {X_test_scaled.shape}")

# Save the scaler for later use (using global config path)
save_scaler_and_encoder(scaler, label_encoder_global, SCALER_SAVE_PATH, ENCODER_SAVE_PATH)

## LSTM Modeli için Veri Hazırlığı

PyTorch LSTM modeli için, veriyi uygun formata dönüştürmemiz gerekir. LSTM modeller sıralı veri bekler, bu nedenle öznitelik vektörünü zamansal bir diziye dönüştüreceğiz.

In [ ]:
# PyTorch tensörlerine dönüştürme ve veri setlerini hazırlama
def create_sequence_data(X, y, sequence_length=SEQUENCE_LENGTH): # Use SEQUENCE_LENGTH from config
    """
    Öznitelik vektörünü sıralı verilere dönüştürür.
    FMA veri seti sıralı yapıda değil, bu nedenle yapay bir sıra oluşturuyoruz.
    """
    # Veri boyutlarını kontrol et
    n_samples, n_features = X.shape
    
    # Veriyi yeniden şekillendirme
    features_per_timestep = n_features // sequence_length
    
    if features_per_timestep == 0:
        print(f"Warning: features_per_timestep is 0. n_features={n_features}, sequence_length={sequence_length}. Adjusting sequence_length.")
        features_per_timestep = 1 # Ensure at least one feature per timestep
        sequence_length = min(sequence_length, n_features) # Adjust sequence length if too large
        print(f"Adjusted: features_per_timestep={features_per_timestep}, sequence_length={sequence_length}")

    # Son timestep'e sığmayan özellikleri ele alma (padding or truncation implicitly handled by slicing)
    # X_padded = np.pad(X, ((0,0), (0, sequence_length * features_per_timestep - n_features)), 'constant') if n_features < sequence_length * features_per_timestep else X[:, :sequence_length * features_per_timestep]
    # X_reshaped = X_padded.reshape(n_samples, sequence_length, features_per_timestep)
    
    # Yeniden şekillendirilmiş veri için array oluşturma
    # Ensure the last dimension matches features_per_timestep
    X_seq = np.zeros((n_samples, sequence_length, features_per_timestep))
    
    # Veriyi yeniden şekillendirme
    for i in range(n_samples):
        for t in range(sequence_length):
            start_idx = t * features_per_timestep
            # Ensure end_idx does not exceed n_features and slice correctly for X_seq's last dimension
            end_idx = start_idx + features_per_timestep 
            current_features_slice = X[i, start_idx:end_idx]
            X_seq[i, t, :len(current_features_slice)] = current_features_slice

    # PyTorch tensörlerine dönüştürme
    X_tensor = torch.FloatTensor(X_seq).to(device) # Move to device
    y_tensor = torch.LongTensor(y).to(device)   # Move to device
    
    return X_tensor, y_tensor

# Sıralı veri için hiperparametre (SEQUENCE_LENGTH is from global config)

# Ölçeklenmiş verileri sıralı forma dönüştürme
X_train_seq, y_train_tensor = create_sequence_data(X_train_scaled, y_train_bal)
X_val_seq, y_val_tensor = create_sequence_data(X_val_scaled, y_val)
X_test_seq, y_test_tensor = create_sequence_data(X_test_scaled, y_test)

print(f"Eğitim veri boyutu (Tensor): {X_train_seq.shape}, Etiketler: {y_train_tensor.shape}")
print(f"Doğrulama veri boyutu (Tensor): {X_val_seq.shape}, Etiketler: {y_val_tensor.shape}")
print(f"Test veri boyutu (Tensor): {X_test_seq.shape}, Etiketler: {y_test_tensor.shape}")

# PyTorch DataLoader oluşturma (BATCH_SIZE from global config)
train_dataset = TensorDataset(X_train_seq, y_train_tensor)
val_dataset = TensorDataset(X_val_seq, y_val_tensor)
test_dataset = TensorDataset(X_test_seq, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"DataLoaders created with batch size: {BATCH_SIZE}")

## 📊 LSTM Model Tanımı ve Eğitimi

Aşağıda müzik türü sınıflandırması için bir LSTM (Long Short-Term Memory) ağı tanımlıyoruz. LSTM'ler, müzik gibi sıralı verilerde başarılı olan bir derin öğrenme mimarisidir.

In [ ]:
# === LSTM Model Definition ===
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_prob, bidirectional=True):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, 
                              batch_first=True, dropout=dropout_prob, 
                              bidirectional=bidirectional)
        
        # Adjust linear layer input if bidirectional
        self.fc_input_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.fc = nn.Linear(self.fc_input_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # x shape: (batch_size, seq_length, input_dim)
        # h0 and c0 are initialized to zero by default if not provided
        out, (hn, cn) = self.lstm(x)
        
        # out shape: (batch_size, seq_length, hidden_dim * num_directions)
        # We are interested in the output of the last time step
        if self.lstm.bidirectional:
            # Concatenate the last hidden state of the forward pass (from hn)
            # and the last hidden state of the backward pass (from hn)
            # hn shape: (num_layers * num_directions, batch, hidden_dim)
            # Forward direction: hn[-2,:,:] (last layer, forward)
            # Backward direction: hn[-1,:,:] (last layer, backward)
            out_forward = hn[-2,:,:] 
            out_backward = hn[-1,:,:]
            out_concat = torch.cat((out_forward, out_backward), dim=1)
        else:
            # If not bidirectional, just take the last hidden state of the last layer
            # hn shape: (num_layers, batch, hidden_dim)
            out_concat = hn[-1,:,:] # (batch, hidden_dim)
            
        out_concat = self.dropout(out_concat)
        out = self.fc(out_concat) # (batch, output_dim)
        return out

# Model Instantiation using global config
input_dim = X_train_seq.shape[2]  # Number of features per timestep
output_dim = len(label_encoder_global.classes_)

model = LSTMClassifier(input_dim, HIDDEN_SIZE, output_dim, NUM_LAYERS, DROPOUT_PROB, BIDIRECTIONAL)
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"LSTM Model Instantiated on {device}:")
print(model)
print(f"Input Dim: {input_dim}, Hidden Dim: {HIDDEN_SIZE}, Output Dim: {output_dim}, Layers: {NUM_LAYERS}, Dropout: {DROPOUT_PROB}, Bidirectional: {BIDIRECTIONAL}")
print(f"Optimizer: Adam, LR: {LEARNING_RATE}")


In [ ]:
# === Model Training Loop ===
print("\nStarting LSTM Model Training...")

# Fix for NumPy 2.0 compatibility - replace np.Inf with np.inf in any imported modules
import numpy as np

# Define EarlyStopping class for NumPy 2.0 compatibility
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Define load_model helper function
def load_model(model, path_param):
    model.load_state_dict(torch.load(path_param))
    return model

# Initialize EarlyStopping with patience and model save path from config
early_stopping = EarlyStopping(patience=PATIENCE_EARLY_STOPPING, verbose=True, path=MODEL_SAVE_PATH)

training_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (sequences, labels) in enumerate(train_loader):
        sequences, labels = sequences.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * sequences.size(0)
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    epoch_loss = running_loss / total_train
    epoch_acc = correct_train / total_train
    training_history['loss'].append(epoch_loss)
    training_history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val)
            val_running_loss += loss_val.item() * sequences_val.size(0)
            _, predicted_val = torch.max(outputs_val.data, 1)
            total_val += labels_val.size(0)
            correct_val += (predicted_val == labels_val).sum().item()

    epoch_val_loss = val_running_loss / total_val
    epoch_val_acc = correct_val / total_val
    training_history['val_loss'].append(epoch_val_loss)
    training_history['val_acc'].append(epoch_val_acc)

    print(f'Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}')

    # Early stopping call (path is now handled within the EarlyStopping class)
    early_stopping(epoch_val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break

print("Training finished.")
# Load best model
# Ensure model is an instance of LSTMClassifier before calling load_model
if isinstance(model, LSTMClassifier):
    model = load_model(model, path_param=MODEL_SAVE_PATH) # Use the specific load_model helper
    print(f"Best model re-loaded from {MODEL_SAVE_PATH} using helper function.")
else:
    # Fallback to direct loading if model object type is unexpected, though it should be LSTMClassifier
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    print(f"Best model loaded from {MODEL_SAVE_PATH} directly.")

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(training_history['loss'], label='Training Loss')
plt.plot(training_history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(training_history['acc'], label='Training Accuracy')
plt.plot(training_history['val_acc'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# === Model Evaluation on Test Set ===
print("\nEvaluating model on the test set...")
model.eval()
y_pred_list = []
y_true_list = []
y_prob_list = [] # For ROC-AUC

with torch.no_grad():
    for sequences_test, labels_test in test_loader:
        sequences_test, labels_test = sequences_test.to(device), labels_test.to(device)
        outputs_test = model(sequences_test)
        
        # Probabilities for ROC-AUC
        probs = torch.softmax(outputs_test, dim=1)
        y_prob_list.extend(probs.cpu().numpy())
        
        _, predicted_test = torch.max(outputs_test.data, 1)
        y_pred_list.extend(predicted_test.cpu().numpy())
        y_true_list.extend(labels_test.cpu().numpy())

y_pred_np = np.array(y_pred_list)
y_true_np = np.array(y_true_list)
y_prob_np = np.array(y_prob_list)

# Classification Report and Confusion Matrix
print("\nTest Set Evaluation:")
plot_confusion_matrix_and_report(y_true_np, y_pred_np, label_encoder_global.classes_, title='Test Set Confusion Matrix')

# Advanced Metrics
print("\nAdvanced Test Set Metrics:")
# Pass the number of classes for num_classes_for_roc
advanced_metrics(y_true_np, y_pred_np, y_prob_np, num_classes_for_roc=len(label_encoder_global.classes_))

# Log Experiment
experiment_params = {
    "K_BEST": K_BEST,
    "SEQUENCE_LENGTH": SEQUENCE_LENGTH,
    "HIDDEN_SIZE": HIDDEN_SIZE,
    "NUM_LAYERS": NUM_LAYERS,
    "DROPOUT_PROB": DROPOUT_PROB,
    "BIDIRECTIONAL": BIDIRECTIONAL,
    "LEARNING_RATE": LEARNING_RATE,
    "EPOCHS_RUN": len(training_history['loss']), # Actual epochs run
    "BATCH_SIZE": BATCH_SIZE,
    "RANDOM_STATE": RANDOM_STATE,
    "MIN_SAMPLES_THRESHOLD": MIN_SAMPLES_THRESHOLD,
    "DATA_USED": "BorderlineSMOTE_balanced"
}

# Collect metrics for logging
# Ensure these are calculated correctly based on your evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_true_np, y_pred_np)
test_f1_weighted = f1_score(y_true_np, y_pred_np, average='weighted')

# Handle ROC-AUC for multi-class
try:
    test_roc_auc_ovr = roc_auc_score(y_true_np, y_prob_np, multi_class='ovr', average='weighted')
except ValueError as e:
    print(f"Could not calculate ROC AUC: {e}")
    test_roc_auc_ovr = None # Or some placeholder like 0.0 or np.nan

experiment_metrics = {
    "test_accuracy": accuracy,
    "test_f1_weighted": test_f1_weighted,
    "test_roc_auc_ovr": test_roc_auc_ovr,
    "final_train_loss": training_history['loss'][-1] if training_history['loss'] else None,
    "final_val_loss": training_history['val_loss'][-1] if training_history['val_loss'] else None,
    "final_train_acc": training_history['acc'][-1] if training_history['acc'] else None,
    "final_val_acc": training_history['val_acc'][-1] if training_history['val_acc'] else None
}

log_experiment(experiment_params, experiment_metrics)

print("\nModel evaluation and logging complete.")

# Save scaler and encoder (already done after scaling, but good to ensure it's here if flow changes)
# save_scaler_and_encoder(scaler, label_encoder_global, SCALER_SAVE_PATH, ENCODER_SAVE_PATH)
# print(f"Scaler and LabelEncoder saved to {SCALER_SAVE_PATH} and {ENCODER_SAVE_PATH} respectively.")


In [ ]:
# === SIMPLE DASHBOARD: Quick Experiment Summary ===
import matplotlib.pyplot as plt

def dashboard(history, metrics):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].plot(history['loss'], label='Loss')
    ax[0].set_title('Training Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    ax[1].plot(history['acc'], label='Accuracy')
    ax[1].set_title('Training Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy')
    ax[1].legend()
    plt.suptitle('Experiment Dashboard')
    plt.tight_layout()
    plt.show()
    print('Final Metrics:', metrics)

# Example usage:
# dashboard(history, {'F1': 0.88, 'ROC-AUC': 0.91})
